# Load Config & Initialize Paths

In [2]:
import tomllib as toml
import os
from pprint import pprint

# Load config
config_path = "Y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\src\\stats\\stats_config.toml"
with open(config_path, 'rb') as f:
    config = toml.load(f)
pprint(config)

# Create paths to save files
analysis_name = config["analysis_name"]
analysis_output_folder = os.path.join(config['paths']["root_save"], analysis_name)
if not os.path.exists(analysis_output_folder):
    os.makedirs(analysis_output_folder)
    print(f"Created folder: {analysis_output_folder}")

os.chdir("Y:\\LabMembers\\MTillman\\GitRepos\\Stroke-R01\\src\\stats")

{'all_factor_columns': ['Subject',
                        'Intervention',
                        'SessionOrder',
                        'Is_Stim',
                        'Frequency',
                        'Intensity',
                        'Speed',
                        'Trial',
                        'Cycle',
                        'Side'],
 'analysis_name': 'Subject_Intervention_Speed',
 'factor_levels_to_keep': {},
 'factor_levels_to_remove': {},
 'factors_levels_order': {'Frequency': ['30', '50'],
                          'Intensity': ['RMT', 'TOL'],
                          'Intervention': ['SHAM1',
                                           'SHAM2',
                                           'RMT30',
                                           'RMT50',
                                           'TOL30',
                                           'TOL50'],
                          'Is_Stim': ['STIM', 'NO_STIM'],
                          'SessionOrder': [1, 2, 3, 4, 

# Load Data

In [3]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(config['paths']['data_file'])

pprint(config['all_factor_columns'])

# Make sure the statistical factors are pd.Categorical
for factor in config['all_factor_columns']:
    if factor in config['factors_levels_order']:
        df[factor] = pd.Categorical(df[factor], 
                                        categories=config['factors_levels_order'][factor], 
                                        ordered=True)
    else:
        df[factor] = pd.Categorical(df[factor])

# Get the list of outcome measure column names
outcome_measures_cols = [col for col in df.columns if col not in config['all_factor_columns']]

# Make sure the outcome measures are numeric
for col in outcome_measures_cols:
    df[col] = pd.to_numeric(df[col])

df.head()

['Subject',
 'Intervention',
 'SessionOrder',
 'Is_Stim',
 'Frequency',
 'Intensity',
 'Speed',
 'Trial',
 'Cycle',
 'Side']


,Subject,Intervention,SessionOrder,Is_Stim,Frequency,Intensity,Speed,Trial,Cycle,Side,...,StepDurations_Sym_PercDiff,StrideDurations_Sym_PercDiff,StepLengths_Sym_Diff,SwingDurations_Sym_Diff,StrideLengths_Sym_Diff,StanceDurations_Sym_Diff,StepWidths_Sym_Diff,StrideWidths_Sym_Diff,StepDurations_Sym_Diff,StrideDurations_Sym_Diff
0,SS01,RMT30,5,STIM,30,RMT,FV,1,1,U,...,9.662871,-0.229726,0.049186,0.073628,0.013025,-0.040075,0.031695,-0.018257,0.074016,-0.002270
1,SS01,RMT30,5,STIM,30,RMT,FV,1,2,A,...,-3.505756,-3.014979,0.018469,-0.044208,0.000087,-0.065018,0.037054,0.011232,-0.026854,-0.029792
2,SS01,RMT30,5,STIM,30,RMT,FV,1,3,U,...,-6.424463,-0.352291,0.030835,-0.096324,0.005565,0.003177,0.038754,0.011661,-0.049210,-0.003481
3,SS01,RMT30,5,STIM,30,RMT,FV,1,4,A,...,-3.705367,0.373747,0.052697,-0.081463,0.003340,0.008514,0.056866,0.013976,-0.028383,0.003693
4,SS01,RMT30,5,STIM,30,RMT,FV,1,5,U,...,-3.705367,1.201761,0.015919,-0.020916,-0.008326,0.027360,0.026810,-0.029810,-0.028383,0.011875


# Process the Data

In [5]:
# Using pymer4
from pymer4.models import Lmer
lmer_formula_base = config['stats']['lmer']['lmer_formula']
lmer_formula = outcome_measures_cols[0] + lmer_formula_base
outcome_measure = outcome_measures_cols[0]
print(lmer_formula)
print('HIP_RMSE_JointAngles_Diff' in df.columns)
model = Lmer(lmer_formula, data=df)
model.fit()
grouping_vars = ['Intervention', 'Speed']
model.post_hoc(outcome_measure, grouping_vars)

HIP_RMSE_JointAngles_Diff~ Intervention * Speed + (1 | Subject)
True


c:\Users\mtillman\AppData\Local\anaconda3\envs\pymer4\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "SessionOrder". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '
c:\Users\mtillman\AppData\Local\anaconda3\envs\pymer4\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "Trial". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when categories are strings.
  warnings.warn('Error while trying to convert '
c:\Users\mtillman\AppData\Local\anaconda3\envs\pymer4\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "Cycle". Fall back to string conversion. The error is: Converting pandas "Category" series to R factor is only possible when c

Linear mixed model fit by REML [’lmerMod’]
Formula: HIP_RMSE_JointAngles_Diff~Intervention*Speed+(1|Subject)

Family: gaussian	 Inference: parametric

Number of observations: 4878	 Groups: {'Subject': 15.0}

Log-likelihood: -11995.160 	 AIC: 24018.319

Random effects:

                 Name    Var    Std
Subject   (Intercept)  1.020  1.010
Residual               7.887  2.808

No random effect correlations specified

Fixed effects:



c:\Users\mtillman\AppData\Local\anaconda3\envs\pymer4\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


pymer4.models.Lmer(fitted = True, formula = HIP_RMSE_JointAngles_Diff~Intervention*Speed+(1|Subject), family = gaussian)


RRuntimeError: Error in emtrends(model, pairwise ~ Intervention | Speed, var = "HIP_RMSE_JointAngles_Diff",  : 
  Variable 'HIP_RMSE_JointAngles_Diff' is not in the dataset


In [13]:
# Using statsmodels
import sys
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(current_dir)
from curr_col_data import curr_col_data
from make_lmer_model import make_lmer_model
vc_formula = None
if "vc_formula" in config['stats']['lmer']:
    vc_formula = config['stats']['lmer']['vc_formula']    
random_effect_factor = config['stats']['lmer']['random_effect_factor']
random_effect_formula_base = config["stats"]["lmer"]["random_effect_formula"]
fixed_effects_formula_base = f"{config['stats']['lmer']['fixed_effects_formula']}"
for col in outcome_measures_cols:

    # Set the random effect formula to have a random slope for the current column
    random_effect_formula = random_effect_formula_base
    if r"{outcome}" in random_effect_formula_base:
        random_effect_formula = random_effect_formula_base.replace(r"{outcome}", col)
    
    # Create a data frame for the current column
    curr_df = curr_col_data(df, col, config['all_factor_columns'])

    # Create the lmer model
    fixed_effects_formula = f"{col} {fixed_effects_formula_base}"
    lmer_model = make_lmer_model(curr_df, fixed_effects_formula, random_effect_factor, random_effect_formula, vc_formula)

    # Get the estimated marginal means

    # Perform the hypothesis testing
    

                     Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   HIP_RMSE_JointAngles_Diff
No. Observations:     4878      Method:               REML                     
No. Groups:           15        Scale:                1.4046                   
Min. group size:      183       Log-Likelihood:       -8661.3625               
Max. group size:      564       Converged:            Yes                      
Mean group size:      325.2                                                    
-------------------------------------------------------------------------------
                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                             0.044    0.740  0.059 0.953 -1.407  1.494
Intervention[T.SHAM2]                -0.019    1.037 -0.019 0.985 -2.053  2.014
Intervention[T.RMT30]                 0.318    1.037  0.307 0

y:\LabMembers\MTillman\GitRepos\Stroke-R01\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
y:\LabMembers\MTillman\GitRepos\Stroke-R01\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
y:\LabMembers\MTillman\GitRepos\Stroke-R01\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                     Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   KNEE_RMSE_JointAngles_Diff
No. Observations:    4878      Method:               REML                      
No. Groups:          15        Scale:                2.1486                    
Min. group size:     183       Log-Likelihood:       -9520.0197                
Max. group size:     564       Converged:            Yes                       
Mean group size:     325.2                                                     
-------------------------------------------------------------------------------
                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                            -0.487    0.642 -0.758 0.448 -1.744  0.771
Intervention[T.SHAM2]                 0.389    0.908  0.428 0.668 -1.390  2.168
Intervention[T.RMT30]                 0.988    0.908  1.088 0

y:\LabMembers\MTillman\GitRepos\Stroke-R01\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                     Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      HAM_RMSE_EMG_Diff
No. Observations:       4878         Method:                  REML             
No. Groups:             15           Scale:                   0.0010           
Min. group size:        183          Log-Likelihood:          9201.7967        
Max. group size:        564          Converged:               Yes              
Mean group size:        325.2                                                  
-------------------------------------------------------------------------------
                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                            -0.007    0.011 -0.631 0.528 -0.030  0.015
Intervention[T.SHAM2]                -0.001    0.016 -0.060 0.952 -0.033  0.031
Intervention[T.RMT30]                 0.012    0.016  0.773 0

y:\LabMembers\MTillman\GitRepos\Stroke-R01\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                     Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      RF_RMSE_EMG_Diff
No. Observations:        4878         Method:                  REML            
No. Groups:              15           Scale:                   0.0008          
Min. group size:         183          Log-Likelihood:          9693.3566       
Max. group size:         564          Converged:               Yes             
Mean group size:         325.2                                                 
-------------------------------------------------------------------------------
                                     Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                            -0.034    0.013 -2.663 0.008 -0.060 -0.009
Intervention[T.SHAM2]                 0.009    0.016  0.562 0.574 -0.022  0.041
Intervention[T.RMT30]                 0.017    0.016  1.073 0

IndexError: index 4871 is out of bounds for axis 0 with size 4871

## Save the Results

In [ ]:
# Save the results

# Plot

In [ ]:
# Plot the results